#Data

Léo Dupire

##Get Dataset

In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
%cd ./data
# %cd /content/gdrive/MyDrive/Projects/Collision-Prediction/data
# !gdown https://drive.google.com/drive/u/0/folders/1h27x1bqewBEqm3cuP8oE9O8F3LOgDPPO
# !unzip Dataset_Student_V2.zip

/scratch/ld2494/collision-prediction/WNet/data


In [3]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import os
import torch
import numpy as np
import tqdm.auto as tqdm
import imageio.v3 as iio
import matplotlib.pyplot as plt

In [5]:
P = "Dataset_Student"

In [9]:
#@title Images
imgs = torch.zeros([1000, 22, 160, 240, 3]) # list(range(1000))
for path in tqdm.tqdm(os.listdir(f"{P}/train/")):
  temp = []
  for i in range(22):
    copy = np.copy(iio.imread(f"{P}/train/{path}/image_{i}.png"))
    temp.append(torch.Tensor(copy))
  imgs[int(path.split("_")[1])] = torch.stack(temp)
torch.save(imgs, 'imgs.pt')

val_imgs = torch.zeros([1000, 22, 160, 240, 3]) # list(range(1000))
for path in tqdm.tqdm(os.listdir(f"{P}/val/")):
  temp = []
  for i in range(22):
    copy = np.copy(iio.imread(f"{P}/val/{path}/image_{i}.png"))
    temp.append(torch.Tensor(copy))
  val_imgs[int(path.split("_")[1]) - 1000] = torch.stack(temp)
torch.save(val_imgs, 'val_imgs.pt')

  0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
#@title Masks
# masks = []
# for path in tqdm.tqdm(os.listdir(f"{P}/train/")):
#   masks.append(torch.Tensor(np.load(f"{P}/train/{path}/mask.npy")))
# masks = torch.stack(masks)
# torch.save(masks, 'masks.pt')

val_masks = list(range(1000))
for path in tqdm.tqdm(os.listdir(f"{P}/val/")):
  val_masks[int(path.split("_")[1]) - 1000] = torch.Tensor(np.load(f"{P}/val/{path}/mask.npy"))
val_masks = torch.stack(val_masks)
torch.save(val_masks, 'val_masks.pt')

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
imgs = torch.load('./imgs.pt')

In [ ]:
print(imgs.shape)
print(val_imgs.shape)

print(masks.shape)
print(val_masks.shape)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(6.4*2, 4.8))
axes[0].imshow(imgs[0][0])
axes[1].imshow(masks[0][0])
plt.show()

In [6]:
dir_list = os.listdir(f"{P}/unlabeled/")
lst1 = [x for x in dir_list if len(x) == 10]
lst2 = [x for x in dir_list if len(x) == 11]
lst1.sort()
lst2.sort()
dirs = lst1 + lst2
dirs

['video_2000',
 'video_2001',
 'video_2002',
 'video_2003',
 'video_2004',
 'video_2005',
 'video_2006',
 'video_2007',
 'video_2008',
 'video_2009',
 'video_2010',
 'video_2011',
 'video_2012',
 'video_2013',
 'video_2014',
 'video_2015',
 'video_2016',
 'video_2017',
 'video_2018',
 'video_2019',
 'video_2020',
 'video_2021',
 'video_2022',
 'video_2023',
 'video_2024',
 'video_2025',
 'video_2026',
 'video_2027',
 'video_2028',
 'video_2029',
 'video_2030',
 'video_2031',
 'video_2032',
 'video_2033',
 'video_2034',
 'video_2035',
 'video_2036',
 'video_2037',
 'video_2038',
 'video_2039',
 'video_2040',
 'video_2041',
 'video_2042',
 'video_2043',
 'video_2044',
 'video_2045',
 'video_2046',
 'video_2047',
 'video_2048',
 'video_2049',
 'video_2050',
 'video_2051',
 'video_2052',
 'video_2053',
 'video_2054',
 'video_2055',
 'video_2056',
 'video_2057',
 'video_2058',
 'video_2059',
 'video_2060',
 'video_2061',
 'video_2062',
 'video_2063',
 'video_2064',
 'video_2065',
 'video_20

In [8]:
#@title Unlabeled Images
unlabeled_imgs = torch.zeros([2600, 21, 160, 240, 3]) #list(range(13000))
for path in tqdm.tqdm(os.listdir(f"{P}/unlabeled/")):
  temp = []
  for i in range(22):
    copy = np.copy(torch.Tensor(iio.imread(f"{P}/unlabeled/{path}/image_{i}.png")))
    temp.append(torch.Tensor(copy))
  unlabeled_imgs[int(path.split("_")[1])-2000] = torch.stack(temp)
torch.save(unlabeled_imgs, 'unlabeled_imgs.pt')

  0%|          | 0/13000 [00:00<?, ?it/s]

/tmp/ipykernel_74764/2379604223.py:6: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  copy = np.copy(torch.Tensor(iio.imread(f"{P}/unlabeled/{path}/image_{i}.png")))


IndexError: index 3547 is out of bounds for dimension 0 with size 2600

In [ ]:
#@title SUBMISSION Images
# P_h = "hidden"

# hidden_imgs = list(range(2000))
# for path in tqdm.tqdm(os.listdir(f"{P_h}/")):
#   temp = []
#   for i in range(11):
#     copy = np.copy(torch.Tensor(iio.imread(f"{P_h}/{path}/image_{i}.png")))
#     temp.append(torch.Tensor(copy))
#   hidden_imgs[int(path.split("_")[1])-15000] = torch.stack(temp)
# hidden_imgs = torch.stack(hidden_imgs).to(int)
# torch.save(hidden_imgs, 'hidden_imgs.pt')

##Visualization

In [ ]:
img = iio.imread("Dataset_Student/train/video_0/image_0.png")
plt.imshow(img)

In [ ]:
plt.hist(img.mean(-1).reshape(-1),bins=256)
plt.show()

In [ ]:
plt.imshow(img.mean(-1),cmap='Greys')
plt.show()